# Setting up the environment

In [ ]:
%%capture
files = !ls
files = [f.split("  ") for f in files][0]

isFRIQML = 'fri_qml' in files
isFRIQMLPath = isFRIQML and "setup.py" in files

# Clone the entire repo. Only run once!
if not isFRIQML:
  !git clone -l -s https://github.com/znajob/fri_qml.git fri_qml

if not isFRIQMLPath:
  %cd fri_qml

!git pull
!pip install -e .

In [ ]:
# MAIN IMPORTS
import pennylane as qml
from pennylane import numpy as np
from friqml.visualisation import plot_quantum_state, plot_histogram
from friqml.utils import eps, random_state_normalized, random_state_unnormalized
from functools import partial
from tqdm.notebook import tqdm
from friqml.utils import eps
from scipy.linalg import expm
import dimod

/usr/local/lib/python3.11/dist-packages/pennylane/capture/capture_operators.py:33: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.4.28. You have version 0.5.2 installed. Please downgrade JAX to <=0.4.28 to avoid runtime errors.
  warnings.warn(


When solving the exercises refer to the [PennyLane documentation](https://pennylane.readthedocs.io/en/stable/).

## Quantum thermalisation

### Exercise 1
The thermal state $\rho=\frac{1}{Z} \sum_n e^{-E_n/T} |n\rangle \langle n|$ is a Boltzmann distribution over the energy eigenvalues $E_n$ of some Hamiltonian $H$. An open quantum system equilibrates with the environment to this state. Equilibration is hard to simulate classically and therefore this process could be exploited for calculations. Using dimod, create a random Ising model over 5 spins.

Using `dimod.SimulatedAnnealingSampler` sample the model at near-zero temperature (0.01) and at a high temperature (100). Write the list of energies in two arrays, `energies_low` and `energies_high`. Note that $\beta$ denotes the inverse temperature $1/T$.

In [ ]:
def random_ising(n=5):
    a = 2*np.random.rand(n)-1
    b = 2*np.random.rand(n, n)-1
    qbm = dimod.BinaryQuadraticModel(a, b, "SPIN")
    return qbm


def sample_model(model):
    n_samples = 100
    sampler = dimod.SimulatedAnnealingSampler()
    sample_low = sampler.sample(model, num_reads=n_samples, beta_range=None)
    sample_high = sampler.sample(
        model, num_reads=n_samples, beta_range=(0.05, 0.01))
    energies_low = [s[1] for s in sample_low.record]
    energies_high = [s[1] for s in sample_high.record]
    return energies_low, energies_high, sample_low

In [ ]:
# TESTS
model = random_ising()
energies_low, energies_high, sample_low = sample_model(model)
print(isinstance(model, dimod.binary_quadratic_model.BinaryQuadraticModel))
print(model.vartype == dimod.SPIN)
print(len(model.variables) == 5)

print(np.isclose(min(energies_low), max(energies_low)))   # This is typicall true, but can sometimes be false
print(np.isclose(min(energies_low), min(energies_high)))  # This is typicall true, but can sometimes be false
print(max(energies_high)-min(energies_high)>1)


True
True
True
False
True
True


### Exercise 2
In QAOA, we approximate the ground state of a target Hamiltonian, starting from the ground state of a mixer Hamiltonian. We can perform the exact same optimization for approximating the thermal state of a target system, starting from the thermal state of another system.

We exploit that if we trace out a subsystem of an entangled system, we end up with a mixed state. We can show that the reduced density matrix of

\begin{align}
| \psi(T) \rangle =1/\sqrt{2 \cosh \frac{1}{2T}} (e^{-1/2T} |+ \rangle \otimes | + \rangle+ e^{1/2T} |- \rangle \otimes | - \rangle)
\end{align}

is equatl to the Gibbs density matrix $\rho=\frac{1}{Z}e^{- H_0/T}$, where $H_0$ is the mixing Hamiltonian in QAOA, i.e. $H_0=-\sum_i\sigma^{\rm x}$. Note that $\sigma^{\rm x}|\pm\rangle=\pm |\pm\rangle$

The state $|\psi(T)\rangle$ can be built with a circuit composed of RY gates and CNOT gates. Write a template `state_preparation` that prepares the two qubit state corresponding to the temperature $T$. The temperature `T` is an argument of the template.

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=None)

In [ ]:
def state_preparation(T=0.1, wires=[0, 1]):
    w0, w1 = wires
    theta = np.arctan2(np.exp(0.5/T), np.exp(-0.5/T))
    qml.RY(2*theta, wires=w0)
    qml.CNOT(wires=wires)
    qml.Hadamard(wires=w0)
    qml.Hadamard(wires=w1)

In [ ]:
@qml.qnode(dev)
def circuit(T=1):
  state_preparation(T=T, wires=[0,1])
  return qml.state()

In [ ]:
# TESTS
print(np.allclose(circuit(1e4),[ 7.0710678e-01+0.j, -3.5355339e-05+0.j, -3.5355339e-05+0.j, 7.0710678e-01+0.j]))
print(np.allclose(circuit(0.01),[ 0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j]))

True
True


### Exercise 3

Write a function `reduced_density` with the argument `rho`. The function should return reduced densities of the matrix `rho` for the first and the second qubit.


In [ ]:
def reduced_density(rho):
    # rho = np.einsum("i,j->i,j",state,np.conj(state))
    rho1 = np.array([[rho[0, 0]+rho[1, 1], rho[0, 2]+rho[1, 3]],
                    [rho[2, 0]+rho[3, 1], rho[2, 2]+rho[3, 3]]])
    rho2 = rho[:2, :2]+rho[2:, 2:]
    return rho1, rho2

In [ ]:
# TESTS
A = np.random.rand(2,2)
B = np.random.rand(2,2)
r1,r2 = reduced_density(np.kron(A,B))
print(np.allclose(r1/np.trace(B),A))
print(np.allclose(r2/np.trace(A),B))

True
True


**Note:** We can now check that the reduced density matrix of the state from exercise 2 is indeed a thermal state of $\sigma^{\rm x}$.

In [ ]:
T=0.4
sx = np.array([[0,1],[1,0]])
rho_gibbs =expm(-sx/T)/np.trace(expm(-sx/T))

In [ ]:
state = circuit(T)

In [ ]:
rho = np.einsum('i,j->ij',state,np.conj(state))
r1,r2 = reduced_density(rho)

In [ ]:
print(np.allclose(r1,rho_gibbs))
print(np.allclose(r2,rho_gibbs))

True
True


# Qboost

Before starting the exercises we prepare a simple Swiss roll dataset.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mpl_toolkits.mplot3d.axes3d as p3
import sklearn.datasets

np.random.seed(0)
data, t = sklearn.datasets.make_swiss_roll(n_samples=500)
labels = np.array(t<t.mean())
idx = np.arange(len(labels))
np.random.shuffle(idx)
# train on a random 2/3 and test on the remaining 1/3
idx_train = idx[:2*len(idx)//3]
idx_test = idx[2*len(idx)//3:]
X_train = data[idx_train]
X_test = data[idx_test]

y_train = 2 * labels[idx_train] - 1  # binary -> spin
y_test = 2 * labels[idx_test] - 1

scaler = sklearn.preprocessing.StandardScaler()
normalizer = sklearn.preprocessing.Normalizer()

X_train = scaler.fit_transform(X_train)
X_train = normalizer.fit_transform(X_train)

X_test = scaler.fit_transform(X_test)
X_test = normalizer.fit_transform(X_test)
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
for l in np.unique(labels):
    ax.scatter(data[labels == l, 0], data[labels == l, 1], data[labels == l, 2])

plt.show()

<Figure size 640x480 with 0 Axes>

### Exercise 1
This exercise is related to random forest (standard machine learning). Random forests do bagging: the algorithm trains many simple decision trees on various subsets of the training data, then averages their output to make predictions. Train a random forest on the above data with 20 trees in the ensemble. Save the trained model in an object called `rf_model`.

In [ ]:
import dimod
import sklearn
from sklearn.ensemble import RandomForestClassifier
metric = sklearn.metrics.accuracy_score
from tqdm.notebook import tqdm

In [ ]:
def train_rf_model(X_train, y_train):
    return RandomForestClassifier(n_estimators=20, random_state=0).fit(X_train, y_train)

In [ ]:
rf_model = train_rf_model(X_train, y_train)

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
metric = sklearn.metrics.accuracy_score
###
### YOUR CODE HERE
###

Note: observe the generalisation gap for the random forest ensemble

In [ ]:
metric(y_train, rf_model.predict(X_train)) - metric(y_test, rf_model.predict(X_test))

0.16762870655086226

### Exercise 2

The random forest as you trained it gives you twenty weak estimators. Write a function `get_qubo` that takes a numpy array of predictions over all weak estimators, the true labels, and a $\lambda$ parameter for regularization, and returns a quadratic binary optimization of the QBoost objective function for dimod

\begin{align}
\mathrm{argmin}_{w} \sum_{k\neq1}^{K}  w_kw_l \left(\sum_{i=1}^{N}\frac{1}{K^2}h_k(x_i)h_l(x_i)\right) + \sum_{k=1}^{K}w_k \left(\frac{N}{K^2} +\lambda-2\sum_{i=1}^{N} \frac{1}{K}h_k(x_i)y_i   \right), \mathrm{w}_k \in \{0,1\}.
\end{align}

Your return value is a weight matrix as a dictionary. You can assume that the prediction array's shape is the number of models times the number of training points.



In [ ]:
def get_qubo(predictions, n_models, y_train, lambd):
    q = predictions @ predictions.T/(n_models ** 2)

    qii = len(y_train) / (n_models ** 2) + lambd - \
        2 * predictions @ y_train/(n_models)

    q[np.diag_indices_from(q)] = qii
    Q = {}
    for i in range(n_models):
        for j in range(i, n_models):
            Q[(i, j)] = q[i, j]
    return Q

In [ ]:
predictions = np.array([[-1, 1], [-1, 1]],dtype=np.float64)
y = [-1, 1]
print(get_qubo(predictions,2, y, 1) == {(0, 0): -0.5, (0, 1): 0.5, (1, 1): -0.5})
print(get_qubo(predictions,2, y, 10) == {(0, 0): 8.5, (0, 1): 0.5, (1, 1): 8.5})

True
True


### Exercise 3
Write a function `find_weights` that finds the optimal value for lambda, i.e. the value that reduces the number of trees used while maintaining the accuracy on the training set with the a specified tolerance `rtol`. The arguments of the function are `predictions`, `rf_model`, `X_train`, `y_train`, and `rtol`. The function should return the calculated weights. You can use the function predict in order to construct the prediction matrix.

In [ ]:
def predict(models, weights, X):
    n_data = len(X)
    T = 0
    y = np.zeros(n_data)
    for i, h in enumerate(models):
        y0 = weights[i] * (2*h.predict(X)-1)  # prediction of weak classifier
        y += y0
        T += np.sum(y0)
    y = np.sign(y - T / (n_data*len(models)))
    return y


def solve_lambda(predictions, rf_model, y_train, lam):
    Q = get_qubo(predictions, len(rf_model.estimators_), y_train, lam)
    sampler = dimod.SimulatedAnnealingSampler()
    response = sampler.sample_qubo(Q, num_reads=10)
    weights = list(response.first.sample.values())
    return weights


def find_weights(predictions, rf_model, X_train, y_train, rtol=0.005):
    models = rf_model.predict(X_train)
    n = len(models)
    ws = np.ones(n)
    acc0 = metric(y_train, models)
    acc = acc0
    ls = 0
    for lam in tqdm(np.arange(12, 25, 0.5)):
        weights = solve_lambda(predictions, rf_model, y_train, lam)
        acc1 = metric(y_train, predict(rf_model.estimators_, weights, X_train))
        adiff = abs(acc0-acc1)
        if adiff < rtol:
            acc = acc1
            ws = weights
            ls = lam
    return ws

In [ ]:
# TESTS
rtol=0.005
predictions = 2*np.array([h.predict(X_train) for h in rf_model.estimators_], dtype=np.float64)-1
weights = find_weights(predictions,rf_model,X_train,y_train,rtol=rtol)

print(sum(weights) < 15)
print(np.isclose(metric(y_train, predict(rf_model.estimators_, weights, X_train)), 1.0, rtol=2*rtol))

  0%|          | 0/26 [00:00<?, ?it/s]

True
True


# Quantum kernel

## Introduction

(taken from Peter Wittek course on QML and based on the paper https://iopscience.iop.org/article/10.1209/0295-5075/119/60002/pdf)

Instead of twisting a machine learning algorithm until it only contains subroutines that have quantum variants, we can reverse our thinking and ask: given a piece of quantum hardware and its constraints, can we come up with a new learning method? For instance, interference is a very natural thing to do for example with a Hadamard gate.
For this to work we need to encode both training and test vectors as amplitudes in a state vector built up out of four registers:

$|0\rangle_c|00..0\rangle_m|00..0\rangle_i|0\rangle_a$

The amplitude of such state will be equal to the value of a feature in a training vector or test vector. To do that we use four registers. The first is a single bit, acting as the ancilla ancilla (a), which will code for either a training (a=0) or a test vector (a=1). The second register, in the notebook example a single bit, will code for the m-th training vector. The third register, in the following exercises also reduced to a single bit, codes for the i-th feature. Lastly the class bit (c) codes for class -1 (c=0), or 1 (c=1).
Hence, if after fully encoding all training data and test data into the state $|\psi>$ the state |1010> has coefficient 0.46 :

\begin{align}|\psi\rangle\ = ....+ 0.46|1010\rangle +....\end{align} ,

Then that implies that the second feature (i=1) of the first (m=0) training vector (a=0), which classifies as class 1 (c=1), has the value 0.46. Note, we assume both training vectors and test vector are normalized.

In a more general expression we can write for a fully encoded state (NB we arrange the order of the registers to be consistent with the code below):

\begin{align}|\psi\rangle = \frac{1}{\sqrt{2M}}\sum_{m=0}^{M-1}|y_m\rangle|m\rangle|\psi_{x^m}\rangle|0\rangle + |y_m\rangle|m\rangle|\psi_{\tilde{x}}\rangle|1\rangle\end{align}

with:

$|\psi_{x^m}\rangle = \sum_{i=0}^{N-1}x_i^m|i\rangle, \; |\psi_{\tilde{x}}\rangle = \sum_{i=0}^{N-1}\tilde{x_i}|i\rangle. \quad$ N being equal to the number of features in the the training and test vectors

As the last summation is independent on m, there will M copies of the test vector in the statevector, one for every training vector.

We now only need to apply a Hadamard gate to the ancilla to interfere the test and training instances. By measuring and post-selecting on the ancilla we calculate the probability to get the class -1 for the current test example as

\begin{align}p(\tilde{y}=-1) = \frac{1}{4Mp_{\rm acc}}\sum_{m|y^m=0}|\tilde{\mathbf{x}}-\mathbf{x}^m|^2,\end{align}
where $p_{\text{acc}} = \frac{1}{4M} \sum_{m} \left| \tilde{x} + x_m \right|^2$ is the probability for the successful postselection.

One can think of the presented circuit as implementing a kernel

\begin{align}\kappa(x,x') = 1-\frac{1}{4M}|\mathbf{x}-\mathbf{x}'|^2.\end{align}

and a classifier

\begin{align}\tilde{y}=\mathrm{sgn}\left(\sum_{m=1}^My^m[1-\frac{1}{4M}|\tilde{\mathbf{x}}-\mathbf{x}^m|^2]\right).\end{align}

## Dataset

During next exercises we will assume the following training data set of two vectors, $S = \{(\begin{bmatrix}0 \\ 1\end{bmatrix}, 0), (\begin{bmatrix}\frac{1}{2}\sqrt{2} \\ \frac{1}{2}\sqrt{2}\end{bmatrix}, 1)\}$. We will also assume a single test instance $\begin{bmatrix}1 \\ 0\end{bmatrix}$.


## Exercise 1
Write a template `state_preparation` on four qubits: ancilla, index, data, and class. You can compose the template into four parts.

1.  `prepare_ancilla`:  puts the ancilla and index qubits into a uniform superposition. **Note** This operation already prepares the test example in the ancilla state $|0\rangle$.
2.  `prepare_example_1` prepare the first training instance, by pushing the feature values into the coefficients of the $|0001\rangle$ and $|0101\rangle$, without affecting any of the other coefficients.
3.  `prepare_example_2`: push the feature values of the second training instance into the coefficients of the $|0011\rangle$ and $|0111\rangle$ states. **Note** that loading vectors with amplitude encoding requires to apply a double controlled rotation by an angle determined by the coordinates of the vector.
4. `prepare_class`: perform a conditional flip of the class qubit, such that the coefficients of the second training vector and those of the copy of the test vector have the class bit equal to 1, without affecting any of the other coefficients

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=4, shots=None)

In [ ]:
def prepare_ancilla():
    qml.Hadamard(0)
    qml.Hadamard(1)


def prepare_example_1():
    # qml.ControlledQubitUnitary([[0, 1], [1, 0]], control_wires=[
    #                            0, 1], wires=2, control_values="10")
    qml.ControlledQubitUnitary([[0, 1], [1, 0]], wires=[0,1,2], control_values="10")


def prepare_example_2():
    fi = np.pi/4
    # qml.ControlledQubitUnitary([[np.cos(fi), -np.sin(fi)], [np.sin(fi), np.cos(fi)]],
    #                            control_wires=[0, 1], wires=2, control_values="11")
    qml.ControlledQubitUnitary([[np.cos(fi), -np.sin(fi)], [np.sin(fi), np.cos(fi)]],
                               wires=[0,1,2], control_values="11")


def prepare_class():
    # qml.ControlledQubitUnitary([[0, 1], [1, 0]], control_wires=[
    #                            0, 1], wires=3, control_values="11")
    # qml.ControlledQubitUnitary([[0, 1], [1, 0]], control_wires=[
    #                            0, 1], wires=3, control_values="01")
    qml.ControlledQubitUnitary([[0, 1], [1, 0]], wires=[0,1,3], control_values="11")
    qml.ControlledQubitUnitary([[0, 1], [1, 0]], wires=[0,1,3], control_values="01")


def state_preparation():
    prepare_ancilla()
    prepare_example_1()
    prepare_example_2()
    prepare_class()

In [ ]:
def swap():
  qml.SWAP(wires=[0,3])
  qml.SWAP(wires=[1,2])

@qml.qnode(dev)
def circuit1():
  prepare_ancilla()
  swap()
  return qml.state()

@qml.qnode(dev)
def circuit2():
  prepare_ancilla()
  prepare_example_1()
  swap()
  return qml.state()

@qml.qnode(dev)
def circuit3():
  prepare_ancilla()
  prepare_example_1()
  prepare_example_2()
  swap()
  return qml.state()


def state_preparation():
  prepare_ancilla()
  prepare_example_1()
  prepare_example_2()
  prepare_class()

@qml.qnode(dev)
def circuit4():
  state_preparation()
  swap()
  return qml.state()

In [ ]:
# TESTS 1
print(np.allclose(circuit1(), np.array([0.5 +0.j, 0.5+0.j, 0.5 +0.j, 0.5+0.j, 0.+0.j, 0. +0.j, 0.+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j])))

True


In [ ]:
# TESTS 2
print(np.allclose(circuit2(), np.array([0.5 +0.j, 0. +0.j, 0.5 +0.j, 0.5+0.j, 0.+0.j, 0.5+0.j, 0.+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j])))

False


In [ ]:
# TESTS 3
print(np.allclose(circuit3(), np.array([0.5 +0.j,  0. +0.j,  0.5 +0.j,  np.sqrt(2)/4 + 0.j,  0.+0.j,  0.5+0.j, 0.+0.j,  np.sqrt(2)/4 +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j])))

False


In [ ]:
circuit4(),np.sqrt(2)/4

(array([ 0.5       +0.j,  0.5       +0.j,  0.5       +0.j, -0.35355339+0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.35355339+0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j,
         0.        +0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j]),
 np.float64(0.3535533905932738))

In [ ]:
# TESTS 4
print(np.allclose(circuit4(), np.array([ 0.5 +0.j,  0.  +0.j,  0. +0.j,  0.+0.j, 0.  +0.j,  0.5 +0.j,  0. +0.j,  0.+0., 0.  +0.j,  0.  +0.j,  0.5+0.j,  np.sqrt(2)/4 +0j, 0.  +0.j,  0.  +0.j,  0. +0.j,  np.sqrt(2)/4 +0.j])))

False


## Exercise 2

To get the final evaluation we need to perform a Hadamard on the ancilla and measure it. If we observe a state 0 then we can measure the class qubit and determine the probabilities for each class. Write a function `kernel_circuit` that that prepares the correct state as before applies a Hadamard gate on the ancilla and samples the ancilla and class qubits in the computational basis. Write also a function `postselect` that receives the results of the measurements and calculates the probability to get ancilly 0 and the probability of the test sample to be in the class 0.  

In [ ]:
def e2_kernel_circuit():
    state_preparation()
    qml.Hadamard(wires=0)
    return qml.sample(wires=[0, 3])


def postselect(samples):
    pacc = np.mean(samples[:, 0])
    ps = np.mean(samples[samples[:, 0] == 0][:, 1])
    return pacc, ps

In [ ]:
dev = qml.device('default.qubit', wires=4, shots=10000)

kernel_circuit = qml.qnode(dev)(e2_kernel_circuit)

In [ ]:
# TESTS
# Basis test
samples = kernel_circuit()
pacc, ps = postselect(samples)

print(np.isclose(pacc,0.317,rtol=0.05))
print(np.isclose(ps,0.635,rtol=0.05))


False
False


In [ ]:
pacc,ps

(np.float64(0.4341), np.float64(0.0))